In [1]:
#!/Tsan/bin/python
# -*- coding: utf-8 -*-

In [2]:
# Libraries To Use
from __future__ import division 
import numpy as np
import pandas as pd
import statsmodels.api as sm
import os
from sklearn import linear_model
from datetime import datetime,time,date
import matplotlib.pyplot as plt
import theano.tensor as T
from theano import function
import seaborn as sns
from theano.tensor.shared_randomstreams import RandomStreams

In [3]:
# Import My own library for factor testing
from SingleFactorTest import factorFilterFunctions as ff
#from config import *

In [4]:
import os 

In [5]:
%load_ext line_profiler

In [6]:
path = ff.data_path # path

In [7]:
filenameFCAP = 'LZ_GPA_VAL_A_FCAP.csv'
os.path.split(path)

('D:/cStrategy/Factor', '')

In [8]:
# --------------------------------------- Global Functions to def---------------------------------- #

In [9]:
# top functions to def
def simpleNormalize(narrowedData):
    dataWinsorized = narrowedData.copy()
    dataWinsorizedTrans = dataWinsorized.T
    MAD = 1.483*np.abs(dataWinsorizedTrans-dataWinsorizedTrans.median(skipna=True))
    return ((dataWinsorizedTrans - dataWinsorizedTrans.mean(axis=0, skipna=True))/dataWinsorizedTrans.std(axis=0, skipna=True)).T

In [10]:
# top function
def neutralizeFactor(normalizedFactorDF, normalizedLFCAPDF, datelist):
    factorNeutralized = pd.DataFrame(index=normalizedFactorDF.index, columns=normalizedFactorDF.columns, data=None, dtype = float)
    for date in datelist:
        LFCAPIndice = normalizedLFCAPDF.loc[date].dropna()
        factorIndice = normalizedFactorDF.loc[date].dropna()
        intersectionStocks = list(set(LFCAPIndice.index) & set(factorIndice.index))
        #dummy_Matrix = pd.get_dummies(IndustryDF.loc[date]).T.iloc[:-1]
        #dummy_Matrix = dummy_Matrix[intersectionStocks].append(LFCAPIndice.loc[intersectionStocks])
        try:
            result = sm. OLS(factorIndice.loc[intersectionStocks].T, LFCAPIndice.loc[intersectionStocks].T).fit()
            factorNeutralized.loc[date][intersectionStocks] = result.resid
        except:
            factorNeutralized.loc[date] = np.NaN
    return factorNeutralized

In [11]:
# --------------------------------------- Function Section End ---------------------------------- #

In [12]:
# Data prepared for Neuralization
FCAP1 = np.log10(pd.read_csv(path+filenameFCAP,infer_datetime_format=True,parse_dates=[0],index_col=0))
NormalizedFCAP = simpleNormalize(FCAP1 )

In [13]:
# --------------------------------------- Calculate Forward Adjusted Price ---------------------------------- #

In [14]:
# the necessary files
filenameAdjustFactor='LZ_GPA_CMFTR_CUM_FACTOR.csv'
filenamePirce='LZ_GPA_QUOTE_TCLOSE.csv'

In [15]:
# first is to calculate forward adjusted pice
def calAdjustedPrice():
    # Adjusted factor
    AdjFacBackward=pd.read_csv(path+filenameAdjustFactor,infer_datetime_format=True,parse_dates=[0],index_col=0)
    #AdjFacBackward=AdjFacBackward[:]

    #PriceData to Adjust
    PriceToAdj=pd.read_csv(path+filenamePirce,infer_datetime_format=True,parse_dates=[0],index_col=0)
    #PriceToAdj=PriceToAdj[:]

    #Calculate
    print AdjFacBackward.shape
    print AdjFacBackward.max().shape
    AdjFacforward = AdjFacBackward / AdjFacBackward.max()
    adjustedPrice = (AdjFacforward * PriceToAdj).round(5)
    #adjustedPrice.index.name = 'LZ_GPA_DERI_AdjustedPriceForward_20-d' 
    adjustedPrice.index.name = 'Own_Factor_AdjustedPriceForward-1d'
    print adjustedPrice.index.name 
    adjustedPrice.to_csv(path+'Own_Factor_AdjustedPriceForward-1d.csv',na_rep='NaN',date_format='%Y%m%d')

In [16]:
#AdjFacBackward = pd.read_csv(path+filenameAdjustFactor,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [17]:
#closePrice = pd.read_csv(path+filenameClose,infer_datetime_format=True,parse_dates=[0],index_col=0)

NameError: name 'filenameClose' is not defined

In [18]:
calAdjustedPrice()

(3007, 3330)
(3330L,)
Own_Factor_AdjustedPriceForward-1d


In [11]:
filenameAdjPrice = 'Own_Factor_AdjustedPriceForward-1d.csv'
AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [19]:
# --------------------------------------- Calculate ILLQ Factor(5-days average) ---------------------------------- #

In [20]:
# 
filenameClose = 'LZ_GPA_QUOTE_TCLOSE.csv'
filenameOpen = 'LZ_GPA_QUOTE_TOPEN.csv'
filenameVolume = 'LZ_GPA_QUOTE_TVOLUME.csv'

In [21]:
def calcILLQ(): # rolling_window is set as 5 days
    openPrice = pd.read_csv(path+filenameOpen,infer_datetime_format=True,parse_dates=[0],index_col=0)
    closePrice = pd.read_csv(path+filenameClose,infer_datetime_format=True,parse_dates=[0],index_col=0)
    volume = pd.read_csv(path+filenameVolume,infer_datetime_format=True,parse_dates=[0],index_col=0)
    if openPrice.shape != closePrice.shape:
        print openPrice.shape, closePrice.shape
        print 'data shape is not equal!'
    else:
        newdf = np.abs((closePrice - openPrice)/openPrice)/volume
        newdf = newdf.rolling(min_periods=5,window=5,center=False).mean()
        newdf.index.name = 'Own_Factor_ILLQ-1d'
        newdf.to_csv(path+'Own_Factor_ILLQ-1d.csv',na_rep='NaN',date_format='%Y%m%d')
        return newdf       

In [22]:
#filenameILLQ = 'Own_Factor_ILLQ-1d.csv'
#openPrice = pd.read_csv(path+filenameILLQ,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [23]:
ILLQdf = calcILLQ()

In [24]:
# TO nuetralize DDA20
NormalizedILLQ = simpleNormalize(ILLQdf)
neutralizedILLQ = neutralizeFactor(NormalizedILLQ, NormalizedFCAP, NormalizedILLQ.index)
neutralizedILLQ.index.name = 'Own_Factor_ADJ_ILLQ_1D'
neutralizedILLQ.to_csv(path+neutralizedILLQ.index.name+'.csv',na_rep='NaN',date_format='%Y%m%d')

In [25]:
# --------------------------------------- Calculate FCAP Adjusted Turnover Volume---------------------------------- #

In [26]:
# --------------------------------------- ILLIQ End--------------------------------- #

In [27]:
# REMINDER: Code in this section can be used to neutralize any new factor! Use this to check some size-affected factor!!!

In [28]:
filenameTURNOVER = 'LZ_GPA_QUOTE_TVOLUME.csv'
filenameFCAP = 'LZ_GPA_VAL_A_FCAP.csv'

In [29]:
TURNOVER = pd.read_csv(path+filenameTURNOVER,infer_datetime_format=True,parse_dates=[0],index_col=0)
FCAP1 = np.log10(pd.read_csv(path+filenameFCAP,infer_datetime_format=True,parse_dates=[0],index_col=0))

TURNOVER= simpleNormalize(TURNOVER)

FCAP1 = simpleNormalize(FCAP1 )

datelist = FCAP1.index.tolist()

In [30]:
adjustedTurnOver = neutralizeFactor(TURNOVER , FCAP1, datelist)

In [31]:
adjustedTurnOver.index.name = 'Own_Factor_AdjustedTurnOver-1d'
adjustedTurnOver.to_csv(path+'Own_Factor_AdjustedTurnOver-1d.csv',na_rep='NaN',date_format='%Y%m%d')

In [32]:
# --------------------------------------- Calculate FCAP Adjusted PB ---------------------------------- #

In [33]:
filenamePB = 'LZ_GPA_VAL_PB.csv'
filenameFCAP = 'LZ_GPA_VAL_A_FCAP.csv'
PB = pd.read_csv(path+filenamePB ,infer_datetime_format=True,parse_dates=[0],index_col=0)
PB = simpleNormalize(PB)

In [34]:
adjustedPB = neutralizeFactor(PB , FCAP1, datelist)
adjustedPB.index.name = 'Own_Factor_AdjustedPB-1d'
adjustedPB.to_csv(path+'Own_Factor_AdjustedPB-1d.csv',na_rep='NaN',date_format='%Y%m%d')

In [35]:
pb1 = pd.read_csv(path+'Own_Factor_AdjustedPB-1d.csv',infer_datetime_format=True,parse_dates=[0],index_col=0)

In [36]:
pb1.columns

Index([u'000005.SZ', u'600601.SH', u'600602.SH', u'600651.SH', u'600652.SH',
       u'600653.SH', u'600654.SH', u'600656.SH', u'000004.SZ', u'000002.SZ',
       ...
       u'300652.SZ', u'300653.SZ', u'603488.SH', u'603269.SH', u'603758.SH',
       u'002872.SZ', u'002873.SZ', u'300656.SZ', u'603197.SH', u'603383.SH'],
      dtype='object', length=3330)

In [37]:
# --------------------------------------- Calculate  x-days return volatility  ---------------------------------- #

In [38]:
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'

In [39]:
def calReturnVol(period): # 90days maybe good
    AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
    returnDF = AdjPrice.pct_change()
    newdf = returnDF.rolling(min_periods=20,window=period,center=False).std()
    newdf.index.name = 'Own_Factor_Volatility_%dd' % period
    newdf.to_csv(path+'Own_Factor_Volatility_%dd.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return newdf

In [40]:
calReturnVol(90).tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
Own_Factor_Volatility_90d,,,,,,,,,,,,,,,,,,,,,
2017-05-16,0.014060,0.013176,0.016601,0.019882,0.023213,0.011861,0.0,NaN,0.014479,0.013808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-17,0.014031,0.013156,0.016669,0.020625,0.023218,0.011850,0.0,NaN,0.014405,0.013708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-18,0.014110,0.013165,0.016527,0.020642,0.023214,0.011850,0.0,NaN,0.014389,0.013646,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-19,0.014082,0.013138,0.016610,0.020712,0.023263,0.011845,0.0,NaN,0.014367,0.013610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-22,0.013808,0.013111,0.016685,0.020806,0.023382,0.011807,0.0,NaN,0.014424,0.013669,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# --------------------------------------- Calculate  x-days return Skew ---------------------------------- #

In [42]:
def calReturnSkew(period):
    AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
    returnDF = AdjPrice.pct_change()
    newdf = returnDF.rolling(min_periods=120,window=period,center=False).skew()
    newdf.index.name = 'Own_Factor_Return_Skew_%dD' % period
    newdf.to_csv(path+newdf.index.name+'.csv',na_rep='NaN',date_format='%Y%m%d')
    return newdf

In [43]:
calReturnSkew(250).tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
Own_Factor_Return_Skew_250D,,,,,,,,,,,,,,,,,,,,,
2017-05-16,1.046336,0.779902,-0.484295,-0.804155,-0.062519,1.826635,0.324742,NaN,0.901697,0.978513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-17,1.078530,0.821875,-0.422259,-0.758886,0.041365,1.826635,0.753594,NaN,0.901697,0.980540,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-18,1.216015,0.848010,-0.408484,-0.646619,0.116221,1.826635,1.011247,NaN,0.901697,0.982773,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-19,1.225200,0.846523,-0.408361,-0.648340,0.120582,1.823593,1.031172,NaN,0.901697,0.982002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-22,1.219134,0.843510,-0.417755,-0.657809,0.117679,1.820546,1.025758,NaN,0.899874,0.980298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# --------------------------------------- Calculate  x-days return above 20 days MA  ---------------------------------- #

In [45]:
def calAbove20MA(period):
    AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
    newdf = AdjPrice - AdjPrice.rolling(min_periods=20,window=period,center=False).mean()
    newdf = newdf.rolling(min_periods=20,window=period,center=False).mean()
    newdf.index.name = 'Own_Factor_Above20MA_%dd' % period
    newdf.to_csv(path+'Own_Factor_Above20MA_%dd.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return newdf 

In [46]:
calAbove20MA(20).tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
Own_Factor_Above20MA_20d,,,,,,,,,,,,,,,,,,,,,
2017-05-16,-0.450500,-0.25030,-0.410175,-0.83575,-0.390075,-0.211025,1.953993e-15,NaN,-0.250200,-0.750650,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-17,-0.460800,-0.24715,-0.471825,-0.79685,-0.301400,-0.207550,1.953993e-15,NaN,-0.190950,-0.748425,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-18,-0.468000,-0.24300,-0.529900,-0.75195,-0.230800,-0.200225,1.953993e-15,NaN,-0.139700,-0.751975,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-19,-0.475844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.097825,-0.756508,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# --------------------------------------- Calculate  ARoon  ---------------------------------- #

In [10]:
AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0).iloc[-1000:]
adcopy = AdjPrice.copy()

NameError: name 'filenameAdjPrice' is not defined

In [49]:
adcopy.iloc[:,2].head()

Own_Factor_AdjustedPriceForward-1d
2013-04-12    3.93462
2013-04-15    3.84497
2013-04-16    3.91469
2013-04-17    3.94458
2013-04-18    3.97446
Name: 600602.SH, dtype: float64

In [50]:
def calAroon(data, l=20):
    datacopy = data.copy()
    for i in range(l,len(datacopy)):
        s = datacopy.iloc[i-l:i]
        #print s
        try:
            data.iloc[i] = pd.Timedelta(s.idxmax().date() - s.idxmin().date()).days/l
        except:
            data.iloc[i] = np.NaN
    return data

In [51]:
#aroonData = adcopy.iloc[:,:10].apply(calAroon,l=20)

In [52]:
# --------------------------------------- Aroon End  ---------------------------------- #

In [53]:
# --------------------------------------- Calculate  daily deal Amount(yuan)   ---------------------------------- #

In [54]:
filenameTVolume = 'LZ_GPA_QUOTE_TVOLUME.csv' # 成交量
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'

In [55]:
def calcDDA():
    turnOver = pd.read_csv(path+filenameTVolume,infer_datetime_format=True,parse_dates=[0],index_col=0)
    closePrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
    #volume = pd.read_csv(path+filenameVolume,infer_datetime_format=True,parse_dates=[0],index_col=0)
    if turnOver.shape != closePrice.shape:
        print turnOver.shape, closePrice.shape
        print 'data shape is not equal!'
    else:
        newdf = turnOver * closePrice
        #newdf = newdf.rolling(min_periods=5,window=5,center=False).mean()
        newdf.index.name = 'Own_Factor_DDA-1d'
        newdf.to_csv(path+'Own_Factor_DDA-1d.csv',na_rep='NaN',date_format='%Y%m%d')
        return newdf           

In [56]:
calcDDA()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
Own_Factor_DDA-1d,,,,,,,,,,,,,,,,,,,,,
2005-01-04,1.120256e+04,9.511504e+04,3.648641e+04,4.231575e+04,4.286417e+04,46478.301754,34848.379104,7.744337e+04,13739.751792,1.176296e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,1.104777e+04,1.539230e+05,7.705574e+04,1.189617e+05,3.607527e+04,36450.030077,66542.020171,2.682239e+04,24501.571164,2.070431e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,1.028649e+04,7.519487e+04,9.378221e+04,1.406231e+05,5.684882e+04,30255.485960,33524.838585,8.755773e+03,23666.695054,2.156412e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,4.150195e+04,1.162695e+05,8.110980e+04,5.668555e+04,5.576362e+04,34962.611051,62727.421697,3.152954e+04,17692.454404,2.093654e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-10,1.024396e+05,1.451684e+05,8.156218e+04,6.196563e+04,2.280886e+04,35335.227232,113085.368636,2.329403e+05,15792.835194,8.330268e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-11,2.546677e+04,7.329183e+04,6.653886e+04,7.007041e+04,2.777977e+04,30690.642067,53326.139704,4.126954e+05,51265.468366,7.055445e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-12,1.819887e+04,9.166338e+04,5.598795e+04,3.804263e+04,2.549850e+04,29026.524715,54666.832954,2.388097e+05,22423.051945,8.959669e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-13,2.032021e+04,1.246899e+05,4.048227e+04,1.689614e+05,2.643201e+04,36763.938919,77921.212387,3.383591e+05,49882.869110,1.099860e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-14,1.890111e+04,1.520970e+05,5.239335e+04,1.093032e+05,6.019900e+04,39863.661422,56831.726570,3.200100e+05,42039.785261,9.886799e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
filenameDDA = 'Own_Factor_DDA-1d.csv'
DDAdf = pd.read_csv(path+filenameDDA,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [58]:
def DDAaverage(period):
    DDAmean = DDAdf.rolling(min_periods=20,window=period,center=False).mean()
    DDAmean.index.name = 'Own_Factor_DDA-%dd' % period
    DDAmean.to_csv(path+'Own_Factor_DDA-%dd.csv' % period, na_rep='NaN',date_format='%Y%m%d')
    return DDAmean

In [59]:
DDA20df= DDAaverage(20)

In [60]:
# TO nuetralize DDA20
NormalizedDDA20 = simpleNormalize(DDA20df)
neutralizedDDA20 = neutralizeFactor(NormalizedDDA20, NormalizedFCAP, NormalizedDDA20.index)
neutralizedDDA20.index.name = 'Own_Factor_ADJ_DDA_20D'
neutralizedDDA20.to_csv(path+neutralizedDDA20.index.name+'.csv',na_rep='NaN',date_format='%Y%m%d')

In [61]:
# --------------------------------------- Calculate  annual idiosyncratic volatility(daily updated)   ---------------------------------- #

In [62]:
def cal_factor_return(factor, stkreturn, factorname):
    '''
    To cal the return of the factor by group method( best(1/3) - worst(1/3))
    Output: Dataframe, the factor Return with only one value column
    Input:
    factor: Dataframe, the factor Value df, Note that shift(1) has been used before this function is used
    stkreturn:  Dataframe, the return of all stock in the market. No shift needed.
    factorname:  String, the name of the factor!
    '''
    factorReturn = pd.DataFrame(index=factor.index[1:-1], columns=[factorname], data =None , dtype =float)
    for date in factorReturn.index:
        factorSlice = factor.loc[date].dropna()
        stkreturnSlice = stkreturn.loc[date].dropna()
        intersection = list(set(factorSlice.index) & set(stkreturnSlice.index))
        factorSlice = factorSlice.loc[intersection]
        stkreturnSlice = stkreturnSlice.loc[intersection]
        
        q_min = factorSlice.quantile(0.33)
        q_max = factorSlice.quantile(0.66)
        q_min_univ = factorSlice[factorSlice<=q_min]
        q_max_univ = factorSlice[factorSlice>=q_max]
        q_min_return = (q_min_univ * stkreturnSlice.loc[q_min_univ.index]).sum() / q_min_univ.sum()
        q_max_return = (q_max_univ * stkreturnSlice.loc[q_max_univ.index]).sum() / q_max_univ.sum()
        #print q_min_return
        factorReturn.loc[date] = q_min_return - q_max_return
    return factorReturn

In [63]:
startTime =  datetime.strptime('20100101', '%Y%m%d')
endTime = datetime.strptime('20170328', '%Y%m%d')
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'
filenameFCAP = 'LZ_GPA_VAL_A_FCAP.csv'
filenamePB='LZ_GPA_VAL_PB.csv'  # 市净率
filenameBENCH = 'LZ_GPA_INDXQUOTE_CLOSE.csv'
ZZ500Index = '000905.SH' #   ZZ500 index code

In [64]:
pricedf = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0).loc[startTime:endTime]
FCAPdf  = pd.read_csv(path+filenameFCAP,infer_datetime_format=True,parse_dates=[0],index_col=0).loc[startTime:endTime]
PBdf = pd.read_csv(path+filenamePB,infer_datetime_format=True,parse_dates=[0],index_col=0).loc[startTime:endTime]
benchmarkdf = pd.read_csv(path+filenameBENCH,infer_datetime_format=True,parse_dates=[0],index_col=0)[ZZ500Index].loc[startTime:endTime].pct_change()
returndf = pricedf.pct_change()
FCAPdf = FCAPdf .shift(1)
PBdf = PBdf.shift(1)
FCAPdf.tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
LZ_GPA_VAL_A_FCAP-d,,,,,,,,,,,,,,,,,,,,,
2017-03-22,631611.7134,985506.1506,825918.6874,815507.8476,756457.5094,670610.0417,1.316040e+06,NaN,329771.5014,2.056177e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-23,623050.7113,972336.8034,825918.6874,797829.5436,732637.1453,668863.6614,1.316040e+06,NaN,330518.1539,2.045498e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-24,619245.8215,965752.1298,825918.6874,793986.4341,721048.8600,661878.1401,1.316040e+06,NaN,324296.0501,2.082389e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-27,621148.2664,974531.6946,825918.6874,800135.4094,708816.7811,665370.9008,1.316040e+06,NaN,319982.0581,2.086272e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-28,616392.1541,976726.5858,825918.6874,803209.8970,703666.4321,661878.1401,1.316040e+06,NaN,315502.1434,2.058322e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
SizeReturn = cal_factor_return(FCAPdf, returndf , 'Size_Return')

In [66]:
SizeReturn.isnull().sum()

Size_Return    0
dtype: int64

In [67]:
PBReturn = cal_factor_return(PBdf, returndf , 'PB_Return')

In [68]:
PBReturn.isnull().sum()

PB_Return    0
dtype: int64

In [69]:
benchmarkdf = benchmarkdf.loc[PBReturn.index[0]:PBReturn.index[-1]]

In [70]:
benchmarkdf .isnull().sum()

0

In [71]:
returndf = returndf.loc[PBReturn.index[0]:PBReturn.index[-1]]

In [72]:
startOfMonthList, endOfMonthList = ff.getLastDayOfMonth(PBReturn.index)

In [73]:
len(startOfMonthList)

87

In [74]:
len(endOfMonthList)

87

In [75]:
datetuple = list(zip(startOfMonthList,endOfMonthList))

In [76]:
def calIdoVol(returnDF , factorReturnList , datetuple ,torelance = 0.05):
    '''
    function to calculate to idiosyncratic volatility.
    Output: Dataframe, The idiosyncratic volatility factor (same shape as ohter daily factor).
    Input : 
    returnDF: Dataframe, the data of the return of all stocks.
    factorReturn: List, the element is the factor-return dataframe. (Obtained by group-method so there is only one value \
    column(and one index column)
    for each dataframe).Note that this df has been shift(1) to ensure that future data is not used!
    datatuple: List, element is the tuple which is consisted of startTime and endTime. Usually zip by the startOfMonthList and endOfMonthList.
    torelance: float, to filter the Nan Value.
    '''
    startdf = pd.DataFrame()
    for i in datetuple:        
        returnDFSlice = returnDF.loc[i[0]:i[1]]
        tempdf= pd.DataFrame(index = returnDFSlice.index, columns = returnDFSlice.columns ,data = None ,dtype =float)
        newReturnSlice = returnDFSlice.loc[:,returnDFSlice.isnull().sum() < returnDFSlice.shape[1] * torelance]
        newReturnSlice = newReturnSlice.fillna(method = 'ffill').fillna(method = 'bfill')
        filterList = newReturnSlice.columns.tolist()
        mapfunction = map(lambda x: x.loc[i[0]:i[1]], factorReturnList)
        totaldf = pd.concat(mapfunction, axis=1) 
        for stk in filterList:
            result = sm.OLS(newReturnSlice[stk],totaldf).fit()
            tempdf[stk].loc[i[0]:i[1]] = np.std(result.resid) * np.sqrt(242)
        startdf = startdf.append(tempdf)
    return startdf        

In [77]:
dfList = [PBReturn,SizeReturn,benchmarkdf]

In [78]:
dd = calIdoVol(returndf , dfList,datetuple)

In [79]:
dd.index.name = 'Own_FACTOR_Idiosyncratic_Volatility'
dd.head()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
Own_FACTOR_Idiosyncratic_Volatility,,,,,,,,,,,,,,,,,,,,,
2010-01-05,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-11,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
dd.to_csv(path+'Own_Factor_Idiosyncratic_Volatility.csv', na_rep='NaN',date_format='%Y%m%d')

In [81]:
# --------------------------------------- Calculate  annual idiosyncratic volatility(daily updated)   ---------------------------------- #

In [82]:
ddc =dd.tail(100)

In [83]:
filenameIDIVOL = 'Own_Factor_Idiosyncratic_Volatility.csv'
idio = pd.read_csv(path+filenameIDIVOL,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [84]:
idio.head()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
Own_FACTOR_Idiosyncratic_Volatility,,,,,,,,,,,,,,,,,,,,,
2010-01-05,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-11,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
dd.head()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
Own_FACTOR_Idiosyncratic_Volatility,,,,,,,,,,,,,,,,,,,,,
2010-01-05,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-11,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
def adj_boxplot(factorData):
    '''To calculate  adjusted -boxplot winsorized data and then Normalize the outcome
    Output: Dataframe, the winsorized and normalized data
    Input: 
    factorData:Dataframe, raw data, can contain nan value
    '''
    copyData = factorData.copy()
    for i in copyData.index:
        temp = copyData.loc[i]
        x = temp.dropna().values
        if len(x) > 0:
            mc = sm.stats.stattools.medcouple(x)
            x.sort()
            q1 = x[int(0.25*len(x))]
            q3 = x[int(0.75*len(x))]
            iqr = q3-q1
            if mc >= 0:
                l = q1-1.5*np.exp(-3.5*mc)*iqr
                u = q3+1.5*np.exp(4*mc)*iqr
            else:
                l = q1-1.5*np.exp(-4*mc)*iqr
                u = q3+1.5*np.exp(3.5*mc)*iqr
            temp.loc[temp < l] = l
            temp.loc[temp > u] = u
            #factor_data.loc[i] = (temp-temp.mean())/temp.std()
    Trans = copyData.T
    return ((Trans  - Trans .mean(axis=0, skipna=True))/Trans .std(axis=0, skipna=True)).T

In [87]:
%mprun  addd = adj_boxplot(dd)

ERROR:root:Line magic function `%mprun` not found.


In [88]:
%lprun -f adj_boxplot adj_boxplot(idio)

In [89]:
# --------------------------------------- Calculate  some random factor  ---------------------------------- #

In [90]:
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'
filenameVolume = 'LZ_GPA_QUOTE_TVOLUME.csv'
closePrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
tradVol = pd.read_csv(path+filenameVolume,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [91]:
def calrandfac(pricedf, factordf):
    sgndf= np.sign(pricedf.pct_change())
    tempo = factordf * sgndf
    tempo = tempo.ewm(ignore_na=True, min_periods=5, halflife = 5).mean()
    return tempo

In [92]:
overheatVol = calrandfac(closePrice, tradVol)

In [93]:
overheatVol

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
LZ_GPA_QUOTE_TVOLUME-d,,,,,,,,,,,,,,,,,,,,,
2005-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-11,17362.211023,15599.700509,8196.017254,7338.990601,1224.180126,6698.553599,10590.838322,-14326.736494,3880.029943,28860.425910,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-12,10402.596662,4943.392692,3961.613465,3901.998182,-714.132059,2198.381750,3396.387402,1668.029096,2044.179410,39472.459598,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-13,11233.659633,12637.857932,1585.294374,9984.066901,970.715897,1740.174325,8768.956296,-15194.558737,3484.987477,50336.580403,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-14,6374.148763,201.505827,-598.890251,3853.326015,-2465.104131,-2078.950153,2830.837631,-27492.719639,1318.541297,24416.840946,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
overheatVol.index.name = 'OVER_HEAT_VOL'
overheatVol.to_csv(path+'Over_Heat_Volume.csv', na_rep='NaN',date_format='%Y%m%d')

In [95]:
# --------------------------------------- Calculate  skewness  ---------------------------------- #
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'
closePrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [96]:
# calculate N-days skewness of the price
def calSkewness(pricedf,period):
    df = pricedf.rolling(min_periods=250,window=period,center=False).skew()
    df.index.name =  'Own_Factor_Skewness_%dd' % period
    df.to_csv(path+'Own_Factor_Skewness_%dd.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return df

In [97]:
priceSkewness = calSkewness(closePrice,250)

In [98]:
# --------------------------------------- Calculate  TurnOver Rate Volatility ---------------------------------- #
filenameTOR= 'LZ_GPA_VAL_TURN.csv'
turnoverdf =  pd.read_csv(path+filenameTOR,infer_datetime_format=True,parse_dates=[0],index_col=0)
FCAP1 = np.log10(pd.read_csv(path+filenameFCAP,infer_datetime_format=True,parse_dates=[0],index_col=0))

In [99]:
def calToRvol(period):
    newdf = turnoverdf.rolling(min_periods=20,window=period,center=False).std()
    newdf.index.name = 'Own_Factor_Turnover_Volatility_%dD' % period
    newdf.to_csv(path+'Own_Factor_Turnover_Volatility_%dD.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return newdf

In [100]:
newsdf = calToRvol(20)

In [101]:
newsdf.tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
Own_Factor_Turnover_Volatility_20D,,,,,,,,,,,,,,,,,,,,,
2017-05-16,0.243625,0.201857,NaN,NaN,1.757617,0.227957,NaN,NaN,NaN,0.101149,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-17,0.244183,0.200327,NaN,0.887374,1.793755,0.234819,NaN,NaN,NaN,0.101016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-18,0.231912,0.199931,NaN,0.888408,1.788389,0.230193,NaN,NaN,NaN,0.104054,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-19,0.239917,0.202112,0.610585,0.888166,1.807533,0.227105,1.742460e-07,NaN,0.0,0.106352,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
NormalizedTOV = simpleNormalize(newsdf )
NormalizedFCAP = simpleNormalize(FCAP1 )

In [103]:
neutralizedTOV = neutralizeFactor(NormalizedTOV, NormalizedFCAP, NormalizedTOV.index)

In [104]:
neutralizedTOV.index.name = 'Own_Factor_ADJ_Turnover_Volatility_20D'
neutralizedTOV.to_csv(path+neutralizedTOV.index.name+'.csv',na_rep='NaN',date_format='%Y%m%d')

In [105]:
def calToRVolD(period):
    newdf = turnoverdf.rolling(min_periods=20,window=period,center=False).mean()
    newdf1 = turnoverdf.rolling(min_periods=500,window=500,center=False).mean()
    newdf = newdf / newdf1 -1
    newdf.index.name = 'Own_Factor_Turnover_Volatility_deviation_%dD' % period
    newdf.to_csv(path+'Own_Factor_Turnover_Volatility_deviation_%dD.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return newdf

In [106]:
df2 = calToRVolD(20)

In [107]:
df2 .tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
Own_Factor_Turnover_Volatility_deviation_20D,,,,,,,,,,,,,,,,,,,,,
2017-05-16,-0.791950,-0.814949,NaN,NaN,-0.191896,-0.878575,NaN,NaN,NaN,-0.804719,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-17,-0.792915,-0.812885,NaN,NaN,-0.161886,-0.886100,NaN,NaN,NaN,-0.803625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-18,-0.803024,-0.816743,NaN,NaN,-0.152835,-0.898041,NaN,NaN,NaN,-0.805967,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-19,-0.807983,-0.820124,NaN,NaN,-0.101979,-0.908283,NaN,NaN,NaN,-0.811706,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
NormalizedTOVD = simpleNormalize(df2)
neutralizedTOVD = neutralizeFactor(NormalizedTOVD, NormalizedFCAP, NormalizedTOVD.index)
neutralizedTOVD.index.name = 'Own_Factor_ADJ_Turnover_Volatility_Deviation_20D'
neutralizedTOVD.to_csv(path+neutralizedTOVD.index.name+'.csv',na_rep='NaN',date_format='%Y%m%d')

In [109]:
neutralizedTOVD .tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
Own_Factor_ADJ_Turnover_Volatility_Deviation_20D,,,,,,,,,,,,,,,,,,,,,
2017-05-16,-0.574129,-0.658868,NaN,NaN,0.338687,-0.721670,NaN,NaN,NaN,-0.942587,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-17,-0.573583,-0.653131,NaN,NaN,0.386787,-0.730623,NaN,NaN,NaN,-0.936982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-18,-0.574223,-0.639371,NaN,NaN,0.412050,-0.733736,NaN,NaN,NaN,-0.893379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-19,-0.571486,-0.632480,NaN,NaN,0.501338,-0.738356,NaN,NaN,NaN,-0.880697,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
# --------------------------------------- Calculate  adjusted Own factor  ILLQ  ---------------------------------- #

In [ ]:
#  --------------------------------------- Calculate 5day price/ 20 day price  ---------------------------------- #

In [111]:
def cal_5_20_position(period):
    AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
    newdf1 = AdjPrice.rolling(min_periods=5,window=5,center=False).mean()
    newdf2 = AdjPrice.rolling(min_periods=20,window=20,center=False).mean()
    newdf = newdf1 / newdf2
    newdf.index.name = 'Own_Factor_5_20_price_deviation_%dD' % period
    newdf.to_csv(path+'Own_Factor_5_20_price_deviation_%dD.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return newdf

In [112]:
cal_5_20_position(1)

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
Own_Factor_Turnover_5_20_price_deviation_1D,,,,,,,,,,,,,,,,,,,,,
2005-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
def cal_5_20_return(period):
    AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
    returndf = AdjPrice.pct_change()
    newdf1 = returndf.rolling(min_periods=5,window=5,center=False).mean()
    newdf2 = returndf.rolling(min_periods=20,window=20,center=False).mean()
    newdf = newdf1 / newdf2
    newdf.index.name = 'Own_Factor_5_20_return_deviation_%dD' % period
    newdf.to_csv(path+'Own_Factor_5_20_return_deviation_%dD.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return newdf

In [136]:
cal_5_20_return(1).tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
Own_Factor_5_20_return_deviation_1D,,,,,,,,,,,,,,,,,,,,,
2017-05-16,0.746786,-0.427015,2.165339,-1.129561,-1.771213,-0.0,NaN,NaN,NaN,-2.014022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-17,-0.245769,-1.539359,1.421817,-2.999003,0.950730,-0.0,NaN,NaN,NaN,-1.553851,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-18,-0.390784,-1.615813,-0.053450,-3.914562,0.823843,-0.0,NaN,NaN,NaN,0.564281,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-19,0.245337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.891471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#  --------------------------------------- Calculate sharpe ,downside risk and so on  ---------------------------------- #

In [10]:
def calsharpe(period):
    returndf = AdjPrice.pct_change()
    annualReturn = (returndf.rolling(min_periods=period,window=period,center=False).mean()+1)**(252/period) - 1 
    annualVol = (returndf.rolling(min_periods=period,window=period,center=False).std()) * np.sqrt(252/period)
    sharpe = annualReturn / annualVol
    sharpe.index.name = 'Own_Factor_sharpe_ratio_%dD' % period
    sharpe.to_csv(path + sharpe.index.name + '.csv',na_rep='NaN', date_format = '%Y%m%d')
    return sharpe

In [11]:
calsharpe(20).tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
Own_Factor_sharpe_ratio_20D,,,,,,,,,,,,,,,,,,,,,
2017-05-16,-1.803191,-1.123105,-1.301375,-0.763498,0.191985,-1.390438,0.0,NaN,NaN,-0.842813,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-17,-1.643653,-1.020492,-1.207710,-0.400169,0.441375,-1.257772,0.0,NaN,NaN,-0.846567,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-18,-1.629873,-0.995893,-1.224239,-0.322203,0.315023,-1.028427,0.0,NaN,NaN,-0.955271,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-19,-1.706517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.004093,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# better calc 250 days downside risk
def caldownsideRisk(period):
    returndf = AdjPrice.pct_change()
    averageReturn = returndf.rolling(min_periods=20,window=period,center=False).mean()
    downsideRisk = (returndf[returndf < averageReturn] .rolling(min_periods=120,window=period,center=False).std()) * np.sqrt(252/period)
    downsideRisk.index.name = 'Own_Factor_downside_risk_%dD' % period
    downsideRisk.to_csv(path + downsideRisk.index.name + '.csv',na_rep='NaN', date_format = '%Y%m%d')
    return downsideRisk

In [40]:
caldownsideRisk(252).tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
Own_Factor_downside_risk_252D,,,,,,,,,,,,,,,,,,,,,
2017-05-16,0.011531,0.011586,0.016497,0.017984,0.016822,NaN,NaN,NaN,0.011854,0.017120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-17,0.011535,0.011617,0.016497,0.018056,NaN,NaN,NaN,NaN,0.011854,0.017103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-18,0.011501,0.011571,0.016479,0.017986,0.016834,NaN,NaN,NaN,0.011854,0.017085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-19,0.011300,0.011328,0.015930,0.017895,NaN,NaN,NaN,NaN,0.011854,0.017114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-22,0.010775,0.011208,0.015697,NaN,NaN,NaN,NaN,NaN,0.011879,0.017142,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
def calsortinoRatio(period):
    returndf = AdjPrice.pct_change()
    annualReturn = (returndf.rolling(min_periods=period,window=period,center=False).mean()+1)**(252/period) - 1 
    filenameDownsideRisk = 'Own_Factor_downside_risk_252D.csv'
    downsideRisk = pd.read_csv(path+filenameDownsideRisk,infer_datetime_format=True,parse_dates=[0],index_col=0)
    sortinoRatio = annualReturn / downsideRisk
    sortinoRatio.index.name = 'Own_Factor_sortino_ratio_%dD' % period
    sortinoRatio.to_csv(path + sortinoRatio.index.name + '.csv',na_rep='NaN', date_format = '%Y%m%d')
    return sortinoRatio

In [42]:
calsortinoRatio(20).tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
Own_Factor_sortino_ratio_20D,,,,,,,,,,,,,,,,,,,,,
2017-05-16,-9.368536,-5.740719,-7.827035,-4.156651,1.380385,NaN,NaN,NaN,0.0,-2.349013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-17,-8.697108,-5.230751,-7.375752,-2.385484,NaN,NaN,NaN,NaN,0.0,-2.362018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-18,-8.618387,-5.086207,-7.473314,-1.903595,2.154638,NaN,NaN,NaN,0.0,-2.653348,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-19,-9.192038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-2.765404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#  --------------------------------------- Calculate return above the industry average return ---------------------------------- #

In [8]:
filenameIndu = 'LZ_GPA_INDU_ZX.csv'   # 每只股票中信行业
finenameInduClose = 'LZ_GPA_INDXQUOTE_CLOSE.csv' # 行业指数收盘价
filenameInduName = 'LZ_GPA_TMP_INDU_ZX.csv' # 行业名字
filenameInduIndex = 'LZ_GPA_TMP_INDEX.csv'  # 行业代码
Indudf = pd.read_csv(path+filenameIndu ,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [30]:
indusReturn = pd.read_csv(path+finenameInduClose ,infer_datetime_format=True,parse_dates=[0],index_col=0).pct_change()
indusIndex = pd.read_csv(path+filenameInduIndex ,infer_datetime_format=True,parse_dates=[0],index_col=0)
indusForstk = pd.read_csv(path+filenameIndu  ,infer_datetime_format=True,parse_dates=[0],index_col=0)
returndf = AdjPrice.pct_change()

In [24]:
indusIndex 

""
LZ_GPA_TMP_INDEX-t
000001.SH
801010.SI
801020.SI
801030.SI
801040.SI
801050.SI
801080.SI
801110.SI
801120.SI


In [22]:
set(indusForstk.iloc[-1].dropna().values)

{1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0}

In [41]:
# get zx index List
indexList = filter(lambda x: x.startswith('CI') and x.endswith('WI'),indusReturn.columns.tolist())

In [42]:
indexList

['CI005001.WI',
 'CI005002.WI',
 'CI005003.WI',
 'CI005004.WI',
 'CI005005.WI',
 'CI005006.WI',
 'CI005007.WI',
 'CI005008.WI',
 'CI005009.WI',
 'CI005010.WI',
 'CI005011.WI',
 'CI005012.WI',
 'CI005013.WI',
 'CI005014.WI',
 'CI005015.WI',
 'CI005016.WI',
 'CI005017.WI',
 'CI005018.WI',
 'CI005019.WI',
 'CI005020.WI',
 'CI005021.WI',
 'CI005022.WI',
 'CI005023.WI',
 'CI005024.WI',
 'CI005025.WI',
 'CI005026.WI',
 'CI005027.WI',
 'CI005028.WI',
 'CI005029.WI']

In [48]:
def fullName(x):
    if np.isnan(x):
        return np.NaN
    else:
        if x<10:
            return 'CI00500'+str(int(x))+'.WI'
        else:
            return 'CI0050'+str(int(x))+'.WI'

In [49]:
sparedf = indusForstk.applymap(fullName)

In [50]:
sparedf

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
LZ_GPA_INDU_ZX-t,,,,,,,,,,,,,,,,,,,,,
2005-01-04,CI005023.WI,CI005025.WI,CI005025.WI,CI005016.WI,CI005002.WI,CI005013.WI,CI005025.WI,CI005018.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,CI005023.WI,CI005025.WI,CI005025.WI,CI005016.WI,CI005002.WI,CI005013.WI,CI005025.WI,CI005018.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,CI005023.WI,CI005025.WI,CI005025.WI,CI005016.WI,CI005002.WI,CI005013.WI,CI005025.WI,CI005018.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,CI005023.WI,CI005025.WI,CI005025.WI,CI005016.WI,CI005002.WI,CI005013.WI,CI005025.WI,CI005018.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-10,CI005023.WI,CI005025.WI,CI005025.WI,CI005016.WI,CI005002.WI,CI005013.WI,CI005025.WI,CI005018.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-11,CI005023.WI,CI005025.WI,CI005025.WI,CI005016.WI,CI005002.WI,CI005013.WI,CI005025.WI,CI005018.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-12,CI005023.WI,CI005025.WI,CI005025.WI,CI005016.WI,CI005002.WI,CI005013.WI,CI005025.WI,CI005018.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-13,CI005023.WI,CI005025.WI,CI005025.WI,CI005016.WI,CI005002.WI,CI005013.WI,CI005025.WI,CI005018.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-14,CI005023.WI,CI005025.WI,CI005025.WI,CI005016.WI,CI005002.WI,CI005013.WI,CI005025.WI,CI005018.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
indusReturn

,000001.SH,801010.SI,801020.SI,801030.SI,801040.SI,801050.SI,801080.SI,801110.SI,801120.SI,801130.SI,...,CI005020.WI,CI005021.WI,CI005022.WI,CI005023.WI,CI005024.WI,CI005025.WI,CI005026.WI,CI005027.WI,CI005028.WI,CI005029.WI
LZ_GPA_INDXQUOTE_CLOSE-d,,,,,,,,,,,,,,,,,,,,,
2005-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,0.007373,0.016469,0.004456,0.006181,0.007887,0.014740,0.019862,0.011064,0.011697,0.019041,...,0.015162,-0.009338,0.020147,0.025150,0.009525,0.017228,0.015825,0.028438,0.038767,0.022877
2005-01-06,-0.009990,-0.008932,-0.021052,-0.015246,-0.013117,-0.024255,-0.009333,0.003823,-0.008801,-0.009018,...,-0.011054,-0.010794,0.001401,-0.000951,-0.003147,-0.008446,-0.015891,-0.001268,-0.000834,-0.006307
2005-01-07,0.004289,0.006396,-0.034633,0.001858,-0.003913,-0.001984,0.005057,0.003139,0.005713,0.003293,...,0.007031,0.000246,0.007886,0.017659,-0.001689,0.017199,0.001250,-0.003203,-0.004097,0.005634
2005-01-10,0.006150,0.010011,0.009066,0.002942,0.008876,0.011004,0.009631,0.005569,0.013586,0.011194,...,0.007863,0.027694,0.011763,0.008656,0.005657,0.008789,0.008307,0.014209,0.014157,0.012255
2005-01-11,0.004041,0.001367,-0.010780,0.001378,0.003865,0.006889,0.002385,0.019664,0.004320,0.002572,...,0.001155,0.006162,0.002429,0.003088,0.006832,-0.002702,-0.001545,0.003778,-0.005191,0.003442
2005-01-12,-0.000429,0.001666,0.007047,0.002129,0.001480,0.010250,-0.006121,-0.007995,0.000291,-0.001719,...,0.000955,-0.004078,-0.007221,-0.002147,-0.001200,-0.001599,0.001271,-0.007357,-0.007121,0.000432
2005-01-13,-0.000485,-0.001884,0.014725,-0.003275,0.002839,-0.003558,0.009279,0.003138,-0.001011,-0.003754,...,-0.001470,-0.002689,-0.006234,0.007541,-0.001782,-0.000699,0.010366,0.002509,0.000851,0.001446
2005-01-14,-0.008512,-0.009840,-0.009507,-0.013344,-0.005467,-0.015186,-0.013204,-0.012613,-0.006748,-0.011508,...,-0.006894,-0.006634,-0.018770,-0.016441,-0.005891,-0.009712,-0.003846,-0.020061,-0.016536,-0.018811


In [125]:
def gg():
    ss = sparedf.copy()
    for date in sparedf.index:
        print date
        indusReturnSlice = indusReturn.loc[date]
        sparedfSlice = sparedf.loc[date]   
        stkList = sparedf.loc[date].dropna().index.tolist() 
        #s = map(lambda x : indusReturnSlice.loc[sparedf.loc[date][x]], stkList)
        #ss.loc[date,stkList] = indusReturnSlice .loc[sparedfSlice.dropna()].values
        ss.set_value(date, stkList, indusReturnSlice .loc[sparedfSlice.dropna()].values)
    return ss

In [126]:
%lprun -f gg gg()

2005-01-04 00:00:00
2005-01-05 00:00:00
2005-01-06 00:00:00
2005-01-07 00:00:00
2005-01-10 00:00:00
2005-01-11 00:00:00
2005-01-12 00:00:00
2005-01-13 00:00:00
2005-01-14 00:00:00
2005-01-17 00:00:00
2005-01-18 00:00:00
2005-01-19 00:00:00
2005-01-20 00:00:00
2005-01-21 00:00:00
2005-01-24 00:00:00
2005-01-25 00:00:00
2005-01-26 00:00:00
2005-01-27 00:00:00
2005-01-28 00:00:00
2005-01-31 00:00:00
2005-02-01 00:00:00
2005-02-02 00:00:00
2005-02-03 00:00:00
2005-02-04 00:00:00
2005-02-16 00:00:00
2005-02-17 00:00:00
2005-02-18 00:00:00
2005-02-21 00:00:00
2005-02-22 00:00:00
2005-02-23 00:00:00
2005-02-24 00:00:00
2005-02-25 00:00:00
2005-02-28 00:00:00
2005-03-01 00:00:00
2005-03-02 00:00:00
2005-03-03 00:00:00
2005-03-04 00:00:00
2005-03-07 00:00:00
2005-03-08 00:00:00
2005-03-09 00:00:00
2005-03-10 00:00:00
2005-03-11 00:00:00
2005-03-14 00:00:00
2005-03-15 00:00:00
2005-03-16 00:00:00
2005-03-17 00:00:00
2005-03-18 00:00:00
2005-03-21 00:00:00
2005-03-22 00:00:00
2005-03-23 00:00:00


In [127]:
finaldf = gg()

2005-01-04 00:00:00
2005-01-05 00:00:00
2005-01-06 00:00:00
2005-01-07 00:00:00
2005-01-10 00:00:00
2005-01-11 00:00:00
2005-01-12 00:00:00
2005-01-13 00:00:00
2005-01-14 00:00:00
2005-01-17 00:00:00
2005-01-18 00:00:00
2005-01-19 00:00:00
2005-01-20 00:00:00
2005-01-21 00:00:00
2005-01-24 00:00:00
2005-01-25 00:00:00
2005-01-26 00:00:00
2005-01-27 00:00:00
2005-01-28 00:00:00
2005-01-31 00:00:00
2005-02-01 00:00:00
2005-02-02 00:00:00
2005-02-03 00:00:00
2005-02-04 00:00:00
2005-02-16 00:00:00
2005-02-17 00:00:00
2005-02-18 00:00:00
2005-02-21 00:00:00
2005-02-22 00:00:00
2005-02-23 00:00:00
2005-02-24 00:00:00
2005-02-25 00:00:00
2005-02-28 00:00:00
2005-03-01 00:00:00
2005-03-02 00:00:00
2005-03-03 00:00:00
2005-03-04 00:00:00
2005-03-07 00:00:00
2005-03-08 00:00:00
2005-03-09 00:00:00
2005-03-10 00:00:00
2005-03-11 00:00:00
2005-03-14 00:00:00
2005-03-15 00:00:00
2005-03-16 00:00:00
2005-03-17 00:00:00
2005-03-18 00:00:00
2005-03-21 00:00:00
2005-03-22 00:00:00
2005-03-23 00:00:00


In [88]:
dates = [pd.Timestamp('2012-05-01'), pd.Timestamp('2012-05-02'), pd.Timestamp('2012-05-03')]
s = pd.DataFrame(index = dates ,data =[[5,6,7],[8,6,0],[1,2,3]],columns=[1,2,3] )
d = pd.DataFrame(index = dates,data =[[1,2,3,1],[2,3,1,2],[3,1,2,3]],columns=['hs','hc','hv','hg'] )
dd =d.copy()

In [129]:
excessReturn = returndf - finaldf 

In [133]:
excessReturn.rolling(min_periods=20,window=20,center=False).mean()
excessReturn.index.name = 'Own_Factor_excess_return_20D'
excessReturn.to_csv(path + excessReturn.index.name + '.csv',na_rep='NaN', date_format = '%Y%m%d')

In [90]:
d

,hs,hc,hv,hg
2012-05-01,1,2,3,1
2012-05-02,2,3,1,2
2012-05-03,3,1,2,3


In [134]:
a=1
a(2)

TypeError: 'int' object is not callable

In [94]:
dd

,hs,hc,hv,hg
2012-05-01,5,6,7,5
2012-05-02,6,0,8,6
2012-05-03,3,1,2,3


In [91]:
for i in dates:
    sSlice = s.loc[i]
    dSlice = d.loc[i]
    dd.loc[i] = sSlice.loc[dSlice].values
print dd

            hs  hc  hv  hg
2012-05-01   5   6   7   5
2012-05-02   6   0   8   6
2012-05-03   3   1   2   3


In [63]:
sparedf.iloc[-10:]

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300652.SZ,300653.SZ,603488.SH,603269.SH,603758.SH,002872.SZ,002873.SZ,300656.SZ,603197.SH,603383.SH
LZ_GPA_INDU_ZX-t,,,,,,,,,,,,,,,,,,,,,
2017-05-09,CI005023.WI,CI005027.WI,CI005025.WI,CI005016.WI,CI005028.WI,CI005013.WI,CI005025.WI,CI005013.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-10,CI005023.WI,CI005027.WI,CI005025.WI,CI005016.WI,CI005028.WI,CI005013.WI,CI005025.WI,CI005013.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-11,CI005023.WI,CI005027.WI,CI005025.WI,CI005016.WI,CI005028.WI,CI005013.WI,CI005025.WI,CI005013.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-12,CI005023.WI,CI005027.WI,CI005025.WI,CI005016.WI,CI005028.WI,CI005013.WI,CI005025.WI,CI005013.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-15,CI005023.WI,CI005027.WI,CI005025.WI,CI005016.WI,CI005028.WI,CI005013.WI,CI005025.WI,CI005013.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-16,CI005023.WI,CI005027.WI,CI005025.WI,CI005016.WI,CI005028.WI,CI005013.WI,CI005025.WI,CI005013.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-17,CI005023.WI,CI005027.WI,CI005025.WI,CI005016.WI,CI005028.WI,CI005013.WI,CI005025.WI,CI005013.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-18,CI005023.WI,CI005027.WI,CI005025.WI,CI005016.WI,CI005028.WI,CI005013.WI,CI005025.WI,CI005013.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-19,CI005023.WI,CI005027.WI,CI005025.WI,CI005016.WI,CI005028.WI,CI005013.WI,CI005025.WI,CI005013.WI,CI005018.WI,CI005023.WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
sparedf